In [1]:
# import pandas module
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import datetime
import yaml

In [2]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.palettes import Turbo256
from bokeh.models import Span
output_notebook()

Loading BokehJS ...

In [3]:
def get_config():
    with open('config.yaml', 'r') as stream:
        config = yaml.safe_load(stream)
    return config

In [4]:
def load_files(subject):
    """
    Function loads the preprocessed files for the given subject; the pre-processing can be found in 'preprocessing_mental_health'
    
    Parameters
        subject:            The subject's initial
    
    Returns
        mental_data:       The dataframe containting the information for the 6 weeeks period
        baseline_data:     The dataframe that is created for the vegan weeks
        vegan_data:        The dataframe that is created for the baseline weeks
    
    """

    # give the path where the needed files are located; with the function get_config
    config = get_config()
    mental_state_files_path = (config['mental_state_files_path'])

    # give the path where the processed files are located
    processed_files_path = mental_state_files_path + '/processed'

    # create the mental file
    mental_filename = processed_files_path + f'/mental_{subject}.csv'

    # create the baseline file
    baseline_filename = processed_files_path + f'/baseline_{subject}.csv'

    # create the vegan file
    vegan_filename = processed_files_path + f'/vegan_{subject}.csv'
    
    # create dataframes for the whole period (mental), baseline and vegan period
    mental_data = pd.read_csv(mental_filename, sep=',')
    baseline_data = pd.read_csv(baseline_filename, sep=',')
    vegan_data = pd.read_csv(vegan_filename, sep=',')
    
    # set 'date' column to datetime 
    mental_data['date'] = pd.to_datetime(mental_data['date'], format='%Y%m%d')
    baseline_data['date'] = pd.to_datetime(baseline_data['date'], format='%Y%m%d')
    vegan_data['date'] = pd.to_datetime(vegan_data['date'], format='%Y%m%d')
    
    return mental_data, baseline_data, vegan_data

In [5]:
def line_plot(subject, mental_data):
    """
    function returns a line plot for the whole 6 weeks including baseline and vegan period
    
    Parameters
        subject:            The subject's initial
        mental_data:        The dataframe with the whole 6 weeks of mental health data
    
    Returns
        line plot for the given subject for over 6 weeks with a separation line at the point where
        the diet was switched.
    
    """
       

    # initiate figure
    p = figure(plot_width=1000, plot_height=400, y_range=(0,7) , x_axis_type='datetime')
    
    # give color list for line plot
    colors = list(Turbo256)
    index = 0
    
    
    for (columnName, columnData) in mental_data.iteritems():
        if index != 0:
            p.line(mental_data['date'], mental_data[columnName], line_width=2, color=colors[index], legend_label=columnName)
        index = index + 1
        
        
    # add the line to see the border between baseline and vegan
    border = time.mktime(datetime(2020, 10, 26, 3, 0, 0).timetuple())*1000
    daylight_savings_start = Span(location=border, dimension='height', line_color='hotpink', line_width=3)
    p.add_layout(daylight_savings_start)

    p.add_layout(p.legend[0], 'right')
    p.legend.click_policy= "hide"

    show(p)
    
    return p

In [6]:
def bar_plot(subject, baseline_data, vegan_data):
    """
    function returns a line plot for the whole 6 weeks including baseline and vegan period
    
    Parameters
        subject:            The subject's initial
    
    Returns
        line plot for the given subject for over 6 weeks with a separation line at the point where
        the diet was switched.
    
    """

    baseline_mean = load_files('a')[1].drop('date', axis=1).mean().round(2)
    vegan_mean = load_files('a')[2].drop('date', axis=1).mean().round(2)
    
    baseline_mean = baseline_mean.to_frame().reset_index()
    vegan_mean = vegan_mean.to_frame().reset_index()

    vegan_baseline = pd.merge(vegan_mean, baseline_mean, on='index')
    vegan_baseline = vegan_baseline.rename(columns={'0_x': 'vegan', '0_y': 'baseline', 'index':'mood'})

    moods = vegan_base['mood'].unique().tolist()

    source = ColumnDataSource(vegan_baseline)

    p = figure(x_range=moods, y_range=(0, 7), title=f"Average moods baseline vs vegan for subject {subject}", plot_width=900)

    p.vbar(x=dodge('mood', -0.2, range=p.x_range), top='baseline', width=0.4, source=source, color="#8AE1FC", legend_label='Baseline')

    p.vbar(x=dodge('mood',  0.2,  range=p.x_range), top='vegan', width=0.4, source=source, color="#C3EB78", legend_label='Vegan')


    p.x_range.range_padding = 0.1
    p.xgrid.grid_line_color = None
    p.legend.location = "top_left"
    p.xaxis.major_label_orientation = "vertical"
    p.legend.click_policy= 'hide'

    show(p)

In [7]:
mental_data = load_files('a')[0]

In [72]:
from bokeh.plotting import figure, show
from bokeh.models import CDSView, ColumnDataSource, CustomJS
from bokeh.models.widgets import Select
from bokeh.layouts import column, layout

data = mental_data
data['active_axis'] = data['relaxed']

source = ColumnDataSource(data=data)

options = []
index = 0

for (columnName, columnData) in mental_data.iteritems():
    if index != 0:
        options.append(columnName)
    index = index + 1 
    
Axesselect = Select(title="Option:", value="relaxed", options=options)
Axesselect.js_on_change('value', CustomJS(args=dict(source=source, Axesselect=Axesselect), code="""
    source.data['active_axis'] = source.data[Axesselect.value];
    source.change.emit();
"""))

# Create a new plot: plot
p = figure(plot_width=1000, plot_height=400, y_range=(0,7) , x_axis_type='datetime')
p.line('date', 'active_axis', source=source)

border = time.mktime(datetime(2020, 10, 26, 3, 0, 0).timetuple())*1000
daylight_savings_start = Span(location=border, dimension='height', line_color='hotpink', line_width=3)
p.add_layout(daylight_savings_start)

controls = [Axesselect]
inputs = column(*controls, width=300)

final_layout = layout([[inputs, p]])
show(final_layout)

In [10]:
# Perform necessary imports
from bokeh.models import ColumnDataSource, Select
 
# Create ColumnDataSource: source
source = ColumnDataSource(data={
    'x' : fertility,
    'y' : female_literacy
})
 
# Create a new plot: plot
plot = figure()
 
# Add circles to the plot
plot.circle('x', 'y', source=source)
 
# Define a callback function: update_plot
def update_plot(attr, old, new):
    # If the new Selection is 'female_literacy', update 'y' to female_literacy
    if new == 'female_literacy': 
        source.data = {
            'x' : fertility,
            'y' : female_literacy
        }
    # Else, update 'y' to population
    else:
        source.data = {
            'x' : fertility,
            'y' : population
        }
 
# Create a dropdown Select widget: select    
select = Select(title="distribution", options=['female_literacy', 'population'], value='female_literacy')
 
# Attach the update_plot callback to the 'value' property of select
select.on_change('value', update_plot)
 
# Create layout and add to current document
layout = row(select, plot)
curdoc().add_root(layout)

NameError: name 'fertility' is not defined